<a href="https://colab.research.google.com/github/mariyajoseph2002/ml/blob/main/deepr1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from tensorflow.keras.layers import Dense

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving mood_depression_dataset1.csv to mood_depression_dataset1.csv


In [ ]:
import pandas as pd
df = pd.read_csv('mood_depression_dataset1.csv')


In [ ]:
# Normalize data
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X = df.drop(columns=["depression_risk"])  # Input features
y = df["depression_risk"]

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'StandardScaler' is not defined

In [ ]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)  # Fit and transform training data
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = tf.keras.Sequential([
    Dense(16, activation="relu", input_shape=(6,)),  # Input layer
    Dense(8, activation="relu"),  # Hidden layer
    Dense(4, activation="softmax")  # Output layer (4 classes: 0-3)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
#the above one is correct
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(X_train.shape[1],))
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3406 - loss: 1.2947 - val_accuracy: 0.5060 - val_loss: 1.0525
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5119 - loss: 1.0317 - val_accuracy: 0.5570 - val_loss: 0.9179
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5614 - loss: 0.9016 - val_accuracy: 0.5740 - val_loss: 0.8373
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5989 - loss: 0.8165 - val_accuracy: 0.5940 - val_loss: 0.7684
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6508 - loss: 0.7489 - val_accuracy: 0.6760 - val_loss: 0.7107
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7005 - loss: 0.6745 - val_accuracy: 0.7160 - val_loss: 0.6512
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7486 - loss: 0.6202 - val_accuracy: 0.7740 - val_loss: 0.5874
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8057 - loss: 0.5572 - val_accuracy: 0.

In [ ]:
feature_means = scaler.mean_
feature_stds = np.sqrt(scaler.var_)  # Standard deviation

print("Means:", feature_means)
print("Standard Deviations:", feature_stds)

Means: [1.76585776 0.84334455 1.10908217 0.39572321 0.87121576 0.6230441 ]
Standard Deviations: [1.01825344 0.82384873 0.86740826 0.48900547 1.02702217 0.26236678]


In [ ]:
model.save("depression_model.h5")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

model_path="/content/drive/My Drive/Colab Notebooks/Data/depression_model.h5"


In [ ]:
model.save(model_path)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

Saved artifact at '/tmp/tmpckrya9qn'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 6), dtype=tf.float32, name='keras_tensor_4')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  138848113855376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138848113855952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138848113856528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138848113857104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138848113854032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138848113858064: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
with open("depression_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved ")


TFLite model saved 


In [ ]:


files.download("depression_model.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print("Shape of X_test_scaled:", X_test_scaled.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_test_scaled: (1000, 6)
Shape of y_test: (1000,)


In [ ]:
import tensorflow as tf

# Convert X_resampled_scaled and y_resampled to tf.data.Dataset
train_ds = tf.data.Dataset.from_tensor_slices((X_resampled_scaled, y_resampled))

# Batch the dataset with a larger batch size (e.g., 1000)
train_ds = train_ds.batch(1000)

# Convert X_test_scaled and y_test to tf.data.Dataset
test_ds = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test))

# Batch the test dataset with a smaller batch size (e.g., 100)
test_ds = test_ds.batch(100)

In [ ]:
import tensorflow_decision_forests as tfdf
from sklearn.metrics import classification_report, f1_score

# Define the RandomForest model
model = tfdf.keras.RandomForestModel(
    random_seed=42,  # For reproducibility
    num_trees=100,  # Number of trees in the forest
    max_depth=10,  # Maximum depth of each tree
    growing_strategy="BEST_FIRST_GLOBAL"  # Tree growing strategy
)

ModuleNotFoundError: No module named 'tensorflow_decision_forests'

In [ ]:
feature_means = scaler.mean_
feature_stds = np.sqrt(scaler.var_)  # Standard deviation

print("Means:", feature_means)
print("Standard Deviations:", feature_stds)